In [8]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi, DriftDetectorApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

In [9]:
SD_IP = "159.65.213.233"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate("admin@seldon.io", "12341234")
    api_client = ApiClient(config)
    return api_client

In [10]:
DEPLOYMENT_NAME = "mab-test"
NAMESPACE = "dev"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"
CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

MODEL_A_NAME = "mock-classifier-a"
MODEL_A_LOCATION = "seldonio/mock_classifier"
MODEL_A_VERSION = "1.6.0-dev"

MODEL_B_NAME = "mock-classifier-b"
MODEL_B_LOCATION = "seldonio/mock_classifier"
MODEL_B_VERSION = "1.6.0-dev"

ROUTER_NAME = "epsilon-greedy-router"
ROUTER_LOCATION = "seldonio/mab_epsilon_greedy"
ROUTER_VERSION = "1.6.0-dev"

ROUTER_BRANCHES = "2"
ROUTER_EPSILON = "0.2"
ROUTER_VERBOSITY = "1"

mldeployment = {
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "fluentd": "true"
        },
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE
    },
    "spec": {
    "name": DEPLOYMENT_NAME,
    "predictors": [
    {
        "name": "default",
        "replicas": 1,
        "componentSpecs": [{
            "spec": {
            "containers": [
                {
                    "image": f"{MODEL_A_LOCATION}:{MODEL_A_VERSION}",
                    "imagePullPolicy": "IfNotPresent",
                    "name": MODEL_A_NAME,
                    "resources": {
                        "requests": {
                            "cpu": CPU_REQUESTS,
                            "memory": MEMORY_REQUESTS
                        },
                        "limits": {
                            "cpu": CPU_LIMITS,
                            "memory": MEMORY_LIMITS
                        }
                    }
                }],
                "terminationGracePeriodSeconds": 20
            }},
            {
            "spec":{
            "containers":[
                {
                    "image": f"{MODEL_B_LOCATION}:{MODEL_B_VERSION}",    
                    "imagePullPolicy": "IfNotPresent",
                    "name": MODEL_B_NAME,
                    "resources": {
                        "requests": {
                            "cpu": CPU_REQUESTS,
                            "memory": MEMORY_REQUESTS
                        },
                        "limits": {
                            "cpu": CPU_LIMITS,
                            "memory": MEMORY_LIMITS
                        }
                    }
                }],
                "terminationGracePeriodSeconds": 20
            }},
            {
            "spec":{
            "containers": [
                {
                    "image": f"{ROUTER_LOCATION}:{ROUTER_VERSION}",   
                    "name": ROUTER_NAME
                }],
                "terminationGracePeriodSeconds": 20
            }}
            ],
        "graph": {
            "name": ROUTER_NAME,
            "type":"ROUTER",
            "parameters": [
            {
                "name": "n_branches",
                "value": ROUTER_BRANCHES,
                "type": "INT"
            },
            {
                "name": "epsilon",
                "value": ROUTER_EPSILON,
                "type": "FLOAT"
            },
            {
                "name": "verbose",
                "value": ROUTER_VERBOSITY,
                "type": "BOOL"
            }
            ],
            "children": [
            {
                "name": MODEL_A_NAME,
                "type":"MODEL",
                "children":[]
            },
            {
                "name": MODEL_B_NAME,
                "type":"MODEL",
                "children":[]
            }]
        },
        "svcOrchSpec": {
            "env": [
                {
                    "name": "SELDON_ENABLE_ROUTING_INJECTION",
                    "value": "true"
                }
            ]
        }}
        ]
    }
}

In [11]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

{'api_version': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'fluentd': 'true'},
              'managed_fields': None,
              'name': 'mab-test',
              'namespace': 'dev',
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'annotations': None,
          'name': 'mab-test',
          'oauth_key': None,
          'oauth_secret': None,
          'predictors': [{'annotations': None,
                          'component_specs': [{'hpa_spec': None,
                                               'keda_spec': None,
            

In [ ]:
# {"data": {"ndarray":[[1.0, 2.0, 5.0]]}}